In [90]:
import numpy as np
import pandas as pd
from mnist import MNIST
from tqdm import tqdm
from collections import Counter

In [2]:
mnist_loader = MNIST('../data/mnist/')
mnist_loader.gz = True

In [3]:
train, y = mnist_loader.load_training()

In [4]:
test, ty = mnist_loader.load_testing()

In [5]:
n_rows = len(train)
t_n_rows = len(test)

In [6]:
y = np.array(y)
ty = np.array(ty)

In [7]:
k = 3

In [69]:
def knn(img_index, k, memfile):
    return np.array(memfile[img_index,:]).argsort()[1:k+1]

## training set

In [13]:
predict = []
for img_index in tqdm(range(n_rows)):
    _y = y[img_index]
    yhat = max(y[knn(img_index, k, fp)])
    predict.append(_y == yhat)

A Jupyter Widget

In [14]:
print("Training accuracy: ", sum(predict)/n_rows * 100, "%", sep="")

Training accuracy: 1.19166666667%


# test set

In [11]:
test_predict = []
for test_index in tqdm(range(t_n_rows)):
    _y = ty[test_index]
    yhat = np.argmax(y[knn(test_index + n_rows, k, fp)])
    test_predict.append(_y == yhat)

100%|██████████| 10000/10000 [00:59<00:00, 167.80it/s]


In [12]:
print("Test set accuracy: ", sum(test_predict)/t_n_rows * 100, "%", sep="")

Test set accuracy: 8.95%


In [29]:
from collections import Counter

In [35]:
Counter(y[knn(5060 + n_rows, 10, fp)]).most_common()

[(0, 8), (6, 1), (3, 1)]

## helpers

In [112]:
batch_size = 5000

In [99]:
def knn_predict(index, k, fp):
    return Counter(y[np.array(fp[index,:]).argsort()[1:k+1]]).most_common()[0][0]

In [115]:
def train_acc(fp, k):
    predict = []
    for batch_index in tqdm(range(n_rows//batch_size)):
        start = batch_index * batch_size
        end = (batch_index + 1) * batch_size
        temp = fp[start:end, :]
        for img_index in range(start, end):
            _y = y[img_index]
            yhat = knn_predict(img_index, k, temp)
            predict.append(_y == yhat)
    return sum(predict)/len(predict)

In [118]:
from sklearn.metrics import accuracy_score

In [134]:
def test_acc(fp, k):
    yhat = []
    for test_index in tqdm(range(t_n_rows)):
        yhat.append(knn_predict(test_index + n_rows, k, fp))
    return accuracy_score(ty, yhat)

## euc without normalization

In [9]:
fp = np.memmap('../data/mnist/tests/edu.memmap', dtype='float32', mode='r', shape=(n_rows + t_n_rows, n_rows))

In [135]:
test_acc(fp, 3)

100%|██████████| 10000/10000 [00:55<00:00, 181.31it/s]


0.96530000000000005

In [137]:
del fp

NameError: name 'fp' is not defined

## ecu 255 normalized

In [138]:
fp = np.memmap('../data/mnist/tests/edu_255.memmap', dtype='float32', mode='r', shape=(n_rows + t_n_rows, n_rows))

In [139]:
test_acc(fp, 3)

100%|██████████| 10000/10000 [01:00<00:00, 166.06it/s]


0.96530000000000005

In [ ]:
del fp